# 毕业项目：为 Arvato Financial Services 金融服务公司实现一个顾客分类报告

该项目要求你分析德国的一家邮购公司的顾客的人口统计数据，将它和一般的人口统计数据进行比较。你将使用非监督学习技术来实现顾客分类，识别出哪些人群是这家公司的基础核心用户。之后，你将把所学的知识应用到第三个数据集上，该数据集是该公司的一场邮购活动的营销对象的人口统计数据。用你搭建的模型预测哪些人更可能成为该公司的顾客。你要使用的数据由我们的合作伙伴 Bertelsmann Arvato Analytics 公司提供。这是真实场景下的数据科学任务。

如果你完成了这个纳米学位的第一学期，做过其中的非监督学习项目，那么你应该对这个项目的第一部分很熟悉了。两个数据集版本不同。这个项目中用到的数据集会包括更多的特征，而且没有预先清洗过。你也可以自由选取分析数据的方法，而不用按照既定的步骤。如果你选择完成的是这个项目，请仔细记录你的步骤和决策，因为你主要交付的成果就是一篇博客文章报告你的发现。

In [13]:
#from google.colab import drive
#drive.mount('/content/drive')
#drive_path = '/content/drive/My Drive/UdacityDataScience/data/'
workInCoLab = True

In [14]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from tqdm import tqdm
tqdm.pandas()

# magic word for producing visualizations in notebook
%matplotlib inline

In [15]:
workInCoLab = True
drive_path = ('../../data/Term2/capstone/arvato_data/', 'C:/data/')[workInCoLab]

## 第 0 部分：了解数据

项目数据中包括四个数据文件

- `Udacity_AZDIAS_052018.csv`: 德国的一般人口统计数据；891211 人（行）x 366 个特征（列）
- `Udacity_CUSTOMERS_052018.csv`: 邮购公司顾客的人口统计数据；191652 人（行）x 369 个特征（列）
- `Udacity_MAILOUT_052018_TRAIN.csv`: 营销活动的对象的人口统计数据；42982 人（行）x 367 个特征（列）
- `Udacity_MAILOUT_052018_TEST.csv`: 营销活动的对象的人口统计数据；42833 人（行）x 366个特征（列）

人口统计数据的每一行表示是一个单独的人，也包括一些非个人特征，比如他的家庭信息、住房信息以及周边环境信息。使用前两个数据文件中的信息来发现顾客（"CUSTOMERS"）和一般人（"AZDIAS"）在何种程度上相同和不同，然后根据你的分析对其余两个数据文件（"MAILOUT"）进行预测，预测更可能成为该邮购公司的客户。

"CUSTOMERS" 文件包括三个额外的列（'CUSTOMER_GROUP'、’'ONLINE_PURCHASE' 和 'PRODUCT_GROUP'），提供了文件中顾客的更多维度的信息。原始的 "MAILOUT" 包括一个额外的列 "RESPONSE"，表示每个收到邮件的人是否成为了公司的顾客。对于 "TRAIN" 子数据集，该列被保留，但是在 "TEST" 子数据集中该列被删除了，它和你最后要在 Kaggle 比赛上预测的数据集中保留的列是对应的。

三个数据文件中其他的所有列都是相同的。要获得关于文件中列的更多信息，你可以参考 Workspace 中的两个 Excel 电子表格。[其一](./DIAS Information Levels - Attributes 2017.xlsx) 是一个所有属性和描述的列表，按照信息的类别进行排列。[其二](./DIAS Attributes - Values 2017.xlsx) 是一个详细的每个特征的数据值对应关系，按照字母顺序进行排列。

在下面的单元格中，我们提供了一些简单的代码，用于加载进前两个数据集。注意，这个项目中所有的 `.csv` 数据文件都是分号(`;`) 分割的，所以 [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) 中需要加入额外的参数以正确地读取数据。而且，考虑数据集的大小，加载整个数据集可能会花费一些时间。

你会注意到在数据加载的时候，会弹出一个警告（warning）信息。在你开始建模和分析之前，你需要先清洗一下数据。浏览一下数据集的结构，查看电子表格中信息了解数据的取值。决定一下要挑选哪些特征，要舍弃哪些特征，以及是否有些数据格式需要修订。我们建议创建一个做预处理的函数，因为你需要在使用数据训练模型前清洗所有数据集。

In [16]:
%%time
azdias = pd.read_csv(drive_path + 'Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv(drive_path + 'Udacity_CUSTOMERS_052018.csv', sep=';')

# use column LNR as index
azdias.set_index('LNR', inplace=True)
customers.set_index('LNR', inplace=True)

Wall time: 41.2 s


In [17]:
azdias.head()

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,0.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,0.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,0.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,0.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [18]:
customers.head()

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,0.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,NaN,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,0.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


## 第0部分：清洗数据 cleaning Data


### 警告形象对应的数据问题

首先我们看看警告所提出的问题，18和19列里到底有什么样的数据问题？

In [19]:
print('column 18 label is', azdias.columns[18])
print('column 19 label is', azdias.columns[19])
print(azdias[azdias.columns[18]].unique())
print(azdias[azdias.columns[19]].unique())

column 18 label isCAMEO_INTL_2015
column 19 label isCJT_GESAMTTYP
[nan 51.0 24.0 12.0 43.0 54.0 22.0 14.0 13.0 15.0 33.0 41.0 34.0 55.0 25.0
 23.0 31.0 52.0 35.0 45.0 44.0 32.0 '22' '24' '41' '12' '54' '51' '44'
 '35' '23' '25' '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45'
 'XX']
[ 2.  5.  3.  4.  1.  6. nan]


In [20]:
def findObjectAttributs(dataframe):
  '''
  find which column in dataframe has object as dtype. 
  Args:
    dataframe {DataFrame} -- it could be customer or azdias
  Returns:
    {set} -- a set of column names, those the dtypes of column is value type 
      object
  '''
  #object_columns = set()
  for attr in dataframe.columns[1:]: 
    attr_unique_values = dataframe[attr].unique()
    if dataframe[attr].dtypes == "object": 
      #object_columns.add(attr)
      print(f'{attr} has value {attr_unique_values}')
  #return object_columns

print('== findObjectAttributs(azdias) ==')
findObjectAttributs(azdias)

print('== findObjectAttributs(customers) ==')
findObjectAttributs(customers)

== findObjectAttributs(azdias) ==
CAMEO_DEU_2015 has value [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C' 'XX']
CAMEO_DEUG_2015 has value [nan 8.0 4.0 2.0 6.0 1.0 9.0 5.0 7.0 3.0 '4' '3' '7' '2' '8' '9' '6' '5'
 '1' 'X']
CAMEO_INTL_2015 has value [nan 51.0 24.0 12.0 43.0 54.0 22.0 14.0 13.0 15.0 33.0 41.0 34.0 55.0 25.0
 23.0 31.0 52.0 35.0 45.0 44.0 32.0 '22' '24' '41' '12' '54' '51' '44'
 '35' '23' '25' '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45'
 'XX']
D19_LETZTER_KAUF_BRANCHE has value [nan 'D19_UNBEKANNT' 'D19_SCHUHE' 'D19_ENERGIE' 'D19_KOSMETIK'
 'D19_VOLLSORTIMENT' 'D19_SONSTIGE' 'D19_BANKEN_GROSS'
 'D19_DROGERIEARTIKEL' 'D19_HANDWERK' 'D19_BUCH_CD' 'D19_VERSICHERUNGEN'
 'D19_VERSAND_REST' 'D19_TELKO_REST' 'D19_BANKEN_DIREKT' 'D19_BANKEN_REST'
 'D19_FREIZEIT' 'D19_LEBENSMITTEL' 'D19_HAUS_DE

居于上面的方向我们发现了这里有 X 和 XX 的数据，对应于数据描述./DIAS Attributes - Values 2017.xlsx这些并不市有效的值。同时还存在string和number共同存在这些列中。我们将先对这两列作相应的处理。同时我们发现作为需要的值是可以处理为-1，表示unknown。

In [21]:
def cleanupCameoDeu2015(dataframe):
  dataframe['CAMEO_DEU_2015'] = dataframe['CAMEO_DEU_2015'].replace('XX', np.nan)
  print(f'after cleanup column CAMEO_DEU_2015 has values: {dataframe["CAMEO_DEU_2015"].unique()}\n')

  dataframe['CAMEO_DEUG_2015'] = dataframe['CAMEO_DEUG_2015']\
                                  .replace('X', np.nan)\
                                  .map(lambda x: str(x)[0])\
                                  .map(lambda x: np.nan if x in ['n'] else x)
  print(f'after cleanup column CAMEO_DEUG_2015 has values: {dataframe["CAMEO_DEUG_2015"].unique()}\n')

  dataframe['CAMEO_INTL_2015'] = dataframe['CAMEO_INTL_2015']\
                              .replace('XX', np.nan)\
                              .map(lambda x: str(x)[0:1] if str(x)[-2:]=='.0' else x)\
                              .map(lambda y: np.nan if y == 'na' else y)
  print(f'after cleanup column CAMEO_INTL_2015 has values: {dataframe["CAMEO_INTL_2015"].unique()}\n')


print("== cleanupCameoDeu2015 in azdias ==")
cleanupCameoDeu2015(azdias)

print("== cleanupCameoDeu2015 in customers ==")
cleanupCameoDeu2015(customers)

== cleanupCameoDeu2015 in azdias ==
after cleanup column CAMEO_DEU_2015 has values: [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C']

after cleanup column CAMEO_DEUG_2015 has values: [nan '8' '4' '2' '6' '1' '9' '5' '7' '3']

after cleanup column CAMEO_INTL_2015 has values: [nan '5' '2' '1' '4' '3' '22' '24' '41' '12' '54' '51' '44' '35' '23' '25'
 '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45']

== cleanupCameoDeu2015 in customers ==
after cleanup column CAMEO_DEU_2015 has values: ['1A' nan '5D' '4C' '7B' '3B' '1D' '9E' '2D' '4A' '6B' '9D' '8B' '5C' '9C'
 '4E' '6C' '8C' '8A' '5B' '9B' '3D' '2A' '3C' '5F' '7A' '1E' '2C' '7C'
 '5A' '2B' '6D' '7E' '5E' '6E' '3A' '9A' '4B' '1C' '1B' '6A' '8D' '7D'
 '6F' '4D']

after cleanup column CAMEO_DEUG_2015 has values: ['1' nan '5' '4' '7' '3' '9' '2' '6' '8']

aft

整个清理数据的目标就是，我们利用pandas的get_dummies方法可以将这个attribute对应成以键值的特征矩阵，后面可以替换对应的attribute列。比如列CAMEO_INTL_2015将被下面的dummies替换。下面是一个例子，dummis将会是一个以LNR为索引，具体attribute__value为列的特征矩阵。

In [22]:
dummies = pd.get_dummies(azdias, columns=['CAMEO_DEU_2015', 'CAMEO_INTL_2015'], prefix_sep='__')
dummies

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,...,CAMEO_INTL_2015__4,CAMEO_INTL_2015__41,CAMEO_INTL_2015__43,CAMEO_INTL_2015__44,CAMEO_INTL_2015__45,CAMEO_INTL_2015__5,CAMEO_INTL_2015__51,CAMEO_INTL_2015__52,CAMEO_INTL_2015__54,CAMEO_INTL_2015__55
LNR,,,,,,,,,,,,,,,,,,,,,
910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,0.0,...,0,0,0,0,0,1,0,0,0,0
910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,0.0,...,0,0,0,0,0,0,0,0,0,0
910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,0.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825761,-1,5.0,17.0,NaN,NaN,NaN,NaN,17.0,15.0,0.0,...,1,0,0,0,0,0,0,0,0,0
825771,-1,9.0,16.0,NaN,NaN,NaN,NaN,16.0,11.0,0.0,...,0,0,0,0,0,1,0,0,0,0
825772,-1,1.0,17.0,NaN,NaN,NaN,NaN,17.0,3.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
del dummies

### Attributes和Values 

Arvato 提供了对应的Meta数据集，Attributes是所有像个的描述。Values包含了每个Attribute可能的数据值，以及其对应的意义Meaning。这里我们把他读取出来，整理成一些相应需要的变量，为后面的数据清洗做准备。特别是Unknown数据，我们可以利用描述来找到对应的值，准备好清洗掉它们。

In [24]:
attributes_df = pd.read_excel(drive_path + 'DIAS Information Levels - Attributes 2017.xlsx', index_col=None, header=1)
values_df = pd.read_excel(drive_path + 'DIAS Attributes - Values 2017.xlsx', index_col=None, header=1)

del attributes_df['Unnamed: 0']
del values_df['Unnamed: 0']

values_df = values_df.fillna(method='ffill', axis=0) # fill merged-cell with first value in above
values_df.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,AGER_TYP,best-ager typology,0,no classification possible
2,AGER_TYP,best-ager typology,1,passive elderly
3,AGER_TYP,best-ager typology,2,cultural elderly
4,AGER_TYP,best-ager typology,3,experience-driven elderly


In [25]:
attributes_df.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [26]:
# display all possilbe meaning has 'unknown'
meaning_se = pd.Series(values_df['Meaning'].unique())
meaning_se[meaning_se.str.contains('known', flags=re.IGNORECASE, regex=True)]

0                                                unknown
10                      unknown / no main age detectable
129                                no transactions known
145                                 no transaction known
199    residental building buildings without actually...
201    mixed building without actually known househol...
202                  company building w/o known company 
203     mixed building without actually known household 
205       mixed building without actually known company 
dtype: object

In [27]:
def setUnknownValueToNan(dataframe):
  ''' 
  calculate the column names of the with meaning 'unknown' or simular meaning.
  the unknown value -1, we will do general replace, so we do ignore it.

  Args:
    None
  Returns:
    {set} of all possible attirbute columns. eg. "D19_VERSI_ANZ_12__0" 
  '''
  unknown = ['unknown', 
            'unknown / no main age detectable',
            'no transactions known',
            'no transaction known']

  attribute_unknown = values_df[values_df['Meaning'].isin(unknown)]
  attribute_unknown['Value'].astype(str).map(lambda st: st.split(', '))
  unknown_columns = dict()

  for index, row in attribute_unknown.iterrows():
    attribute_name = row['Attribute'].replace('_RZ', '')
    attribute_values = str(row['Value']).split(', ')
    for unknown_val in attribute_values:
      if unknown_val == '-1':
        continue
      #unknown_columns.add(f'{attribute_name}__{unknown_val}')
      unknown_columns[attribute_name] = unknown_val
  
  for (key, value) in unknown_columns.items():
    if key in azdias.columns:
        dataframe[key].replace(value, np.nan, inplace=True)
    else:
        print(f'Can not find {key} column，just skip')

In [28]:
setUnknownValueToNan(azdias)

Can not find D19_BUCH column，just skip
Can not find GEOSCORE_KLS7 column，just skip
Can not find HAUSHALTSSTRUKTUR column，just skip
Can not find WACHSTUMSGEBIET_NB column，just skip


attributes 和 values定义和实际数据中使用的差距

In [29]:
azdias_attributes = azdias.columns
customer_attributes = customers.columns
attributes_in_valuesDf = values_df['Attribute'].unique()
attributes_in_attributesDf = attributes_df['Attribute'].unique()

print(f'there are {azdias_attributes.size} in azdias')
print(f'there are {customer_attributes.size} in customers')
print(f'there are {attributes_in_valuesDf.shape[0]} in values_df')
print(f'there are {attributes_in_attributesDf.shape[0]} in attributes_df')

there are 365 in azdias
there are 368 in customers
there are 314 in values_df
there are 313 in attributes_df


为什么在azdias和customer的特征列数量和描述数据values和attributs不对应呢？这里我们进一步通过显示数据来做分析。

In [30]:
attributes_without_meta = set()
ind = 0
for attr in azdias_attributes:
  ind += 1
  if attr not in attributes_in_valuesDf:
    attributes_without_meta.add(attr)
    print(f'Attribute No.{ind} {attr} is not in attributes_df, but in azdias values are: {azdias[attr].unique()}')

Attribute No.2 AKT_DAT_KL is not in attributes_df, but in azdias values are: [nan  9.  1.  5.  8.  7.  6.  4.  3.  2.]
Attribute No.4 ALTER_KIND1 is not in attributes_df, but in azdias values are: [nan 17. 10. 18. 13. 16. 11.  6.  8.  9. 15. 14.  7. 12.  4.  3.  5.  2.]
Attribute No.5 ALTER_KIND2 is not in attributes_df, but in azdias values are: [nan 13.  8. 12. 10.  7. 16. 15. 14. 17.  5.  9. 18. 11.  6.  4.  3.  2.]
Attribute No.6 ALTER_KIND3 is not in attributes_df, but in azdias values are: [nan 10. 18. 17. 16.  8. 15.  9. 12. 13. 14. 11.  7.  5.  6.  4.]
Attribute No.7 ALTER_KIND4 is not in attributes_df, but in azdias values are: [nan 10.  9. 16. 14. 13. 11. 18. 17. 15.  8. 12.  7.]
Attribute No.8 ALTERSKATEGORIE_FEIN is not in attributes_df, but in azdias values are: [nan 21. 17. 13. 14. 10. 16. 20. 11. 19. 15. 18.  9. 22. 12.  0.  8.  7.
 23.  4. 24.  6.  3.  2.  5. 25.  1.]
Attribute No.11 ANZ_KINDER is not in attributes_df, but in azdias values are: [nan  0.  1.  2.  3.  4. 

通过观察我们发现，D19_LETZTER_KAUF_BRANCHE的值刚好对应了其他D19的列，数据看上去有一定的重复。CJT_KATALOGNUTZER也是类似情况，被其他CJT列所重复。ANZ_STATISTISCHE_HAUSHALTE，EXTSEL992有大量的数值，但是我们这里缺乏具体的meta数据，这里我们决定不再保留。EINGEZOGENAM_HH_JAHR。GEBURTSJAHR是出身年份，我们还有其他的列含有相关年龄的列ALTER_HH所以我们也决定忽略。

In [31]:
azdias_shapeBefore = azdias.shape[1]
customers_shapeBefore = customers.shape[1]
columns_to_drop = {'D19_LETZTER_KAUF_BRANCHE', 
                   'CJT_KATALOGNUTZER', 
                   'EINGEZOGENAM_HH_JAHR', 
                   'ANZ_STATISTISCHE_HAUSHALTE',
                   'ANZ_HAUSHALTE_AKTIV',
                   'VERDICHTUNGSRAUM', 
                   'EXTSEL992',
                   'GEBURTSJAHR',
                   'ALTER_KIND1',
                   'ALTER_KIND2',
                   'ALTER_KIND3',
                   'ALTER_KIND4'}

azdias.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)
customers.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)

columns_to_drop.clear()

print(f'before azdias drop {azdias_shapeBefore}, and after {azdias.shape}')
print(f'before customers drop {customers_shapeBefore}, and customers {customers.shape}')

before azdias drop 365, and after (891221, 353)
before customers drop 368, and customers (191652, 356)


In [32]:
# extract special columns from customers
customer_special_columns = ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']
customer_special_df = customers[customer_special_columns]

print(f'columns {customer_special_columns} will be delete from customers')
customers.drop(columns=customer_special_columns, axis=1, errors='ignore', inplace=True)

customer_special_df.head()

columns ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'] will be delete from customers


,CUSTOMER_GROUP,ONLINE_PURCHASE,PRODUCT_GROUP
LNR,,,
9626,MULTI_BUYER,0,COSMETIC_AND_FOOD
9628,SINGLE_BUYER,0,FOOD
143872,MULTI_BUYER,0,COSMETIC_AND_FOOD
143873,MULTI_BUYER,0,COSMETIC
143874,MULTI_BUYER,0,FOOD


EINGEFUEGT_AM是数据添加的时间，一共有5163个时间点，我们只采纳的年份作为特这。将其替换为EINGEFUEGT_AM将会只是数据输入的年份。

In [33]:
def pickYearValue(dataframe, attributeName):
  '''
  use only the year of timestamp as value for the attribute
  Args:
    dataframe {DataFrame} -- customers or azdias
    attributeName {string} -- attribute name, which hast timestamp value 
      like 1992-02-10 00:00:00
  Returns:
    None
  '''
  attr_values = dataframe[attributeName].unique()
  print(f'Attribute {attributeName} has {attr_values.shape[0]} values')
  print('Before change:\n', dataframe[attributeName].head())
  dataframe[attributeName] = dataframe[attributeName]\
                            .map(lambda x: str(x)[:4] if x != np.nan else x)\
                            .map(lambda x: np.nan if x=='nan' else x)
  print(f'After change:\n', dataframe[attributeName].head())
  print('We replaced dataframe[attributeName] with only the year values:',
        dataframe[attributeName].unique())


print("== pickYearValue(customers, 'EINGEFUEGT_AM') ==")
pickYearValue(customers, 'EINGEFUEGT_AM')

print("== pickYearValue(azdias, 'EINGEFUEGT_AM') ==")
pickYearValue(azdias, 'EINGEFUEGT_AM')

== pickYearValue(customers, 'EINGEFUEGT_AM') ==
Attribute EINGEFUEGT_AM has 3035 values
Before change:
LNR
9626      1992-02-12 00:00:00
9628                      NaN
143872    1992-02-10 00:00:00
143873    1992-02-10 00:00:00
143874    1992-02-12 00:00:00
Name: EINGEFUEGT_AM, dtype: object
After change:
LNR
9626      1992
9628       NaN
143872    1992
143873    1992
143874    1992
Name: EINGEFUEGT_AM, dtype: object
We replaced dataframe[attributeName] with only the year values:['1992' nan '2004' '1997' '1995' '2007' '2005' '1996' '2012' '1994' '2008'
 '2003' '2006' '1993' '1998' '2015' '2011' '2000' '1999' '2009' '2010'
 '2002' '2014' '2001' '2013' '2016']
== pickYearValue(azdias, 'EINGEFUEGT_AM') ==
Attribute EINGEFUEGT_AM has 5163 values
Before change:
LNR
910215                    NaN
910220    1992-02-10 00:00:00
910225    1992-02-12 00:00:00
910226    1997-04-21 00:00:00
910241    1992-02-12 00:00:00
Name: EINGEFUEGT_AM, dtype: object
After change:
LNR
910215     NaN
910220    19

#### 在FEIN和GROB数据中做选择

在属性描述中我们看到，有不少属性我们同时拥有细化（FEIN）和粗略（GROB）的数据特征。这里在试验初期我们决定采用粗略的特征。

In [34]:
print(azdias['LP_FAMILIE_FEIN'].unique())
print(azdias['LP_FAMILIE_GROB'].unique())

[ 2.  5.  1.  0. 10.  7. 11.  3.  8.  4.  6. nan  9.]
[ 2.  3.  1.  0.  5.  4. nan]


In [35]:
for attr in attributes_in_attributesDf:
  if attr.endswith('_FEIN'):
    columns_to_drop.add(attr)

print(f'== delete {columns_to_drop} from azdias ==')
azdias.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)

print(f'== delete {columns_to_drop} from customers ==')
customers.drop(columns=columns_to_drop, axis=1, errors='ignore', inplace=True)
columns_to_drop.clear()

== delete {'LP_FAMILIE_FEIN', 'LP_STATUS_FEIN', 'LP_LEBENSPHASE_FEIN'} from azdias ==
== delete {'LP_FAMILIE_FEIN', 'LP_STATUS_FEIN', 'LP_LEBENSPHASE_FEIN'} from customers ==


In [36]:
del values_df
del attributes_df

In [37]:
def findObjectAttributs(dataframe):
  '''
  find which column in dataframe has object as dtype. 
  Args:
    dataframe {DataFrame} -- it could be customer or azdias
  Returns:
    {set} -- a set of column names, those the dtypes of column is value type 
      object
  '''
  object_columns = set()
  for (columnName, columnData) in dataframe.iteritems():
    attr_unique_values = columnData.unique()
    if columnData.dtypes == "object": 
      object_columns.add(columnName)
      print(f'{columnName} has value {attr_unique_values}')
  return object_columns

print('== findObjectAttributs(azdias) ==')
object_dtype_columns = findObjectAttributs(azdias)
print('== findObjectAttributs(customers) ==')
findObjectAttributs(customers)

== findObjectAttributs(azdias) ==
CAMEO_DEU_2015 has value [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C']
CAMEO_DEUG_2015 has value [nan '8' '4' '2' '6' '1' '9' '5' '7' '3']
CAMEO_INTL_2015 has value [nan '5' '2' '1' '4' '3' '22' '24' '41' '12' '54' '51' '44' '35' '23' '25'
 '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45']
EINGEFUEGT_AM has value [nan '1992' '1997' '2005' '2009' '1995' '1996' '2002' '2015' '2004' '2000'
 '2008' '1994' '1993' '2003' '2014' '2016' '2007' '1999' '2010' '2001'
 '1998' '2006' '2013' '2012' '2011' '1991']
OST_WEST_KZ has value [nan 'W' 'O']
== findObjectAttributs(customers) ==
CAMEO_DEU_2015 has value ['1A' nan '5D' '4C' '7B' '3B' '1D' '9E' '2D' '4A' '6B' '9D' '8B' '5C' '9C'
 '4E' '6C' '8C' '8A' '5B' '9B' '3D' '2A' '3C' '5F' '7A' '1E' '2C' '7C'
 '5A' '2B' '6D' '7E' '5E' '6

{'CAMEO_DEUG_2015',
 'CAMEO_DEU_2015',
 'CAMEO_INTL_2015',
 'EINGEFUEGT_AM',
 'OST_WEST_KZ'}

In [38]:
def findAttributesWithMuchValues(dataframe, valueSizeLimit = 10):
  '''
  find attributes/columns in dataframe, they have an oversize of the values. 

  Args:
    dataframe {DataFrame} -- it could be customers and azdias
    valueSizeLimit {int} -- the limitation of value size you want to check, 
      default 10
  Returns:
    {set} -- all attributs, in dataframe has more than 10 values
  '''
  value_oversize_columns = set()
  for (columnName, columnData) in dataframe.iteritems():
    attr_unique_values = columnData.unique()
    if attr_unique_values.size >= valueSizeLimit: 
      value_oversize_columns.add(columnName)
      print(f'{columnName} has value {attr_unique_values}')
  #return value_oversize_columns

print('== findAttributesWithMuchValues(azdias) ==')
findAttributesWithMuchValues(azdias)

print('== findAttributesWithMuchValues(customers) ==')
findAttributesWithMuchValues(customers)

== findAttributesWithMuchValues(azdias) ==
AKT_DAT_KL has value [nan  9.  1.  5.  8.  7.  6.  4.  3.  2.]
ALTER_HH has value [nan  0. 17. 13. 20. 10. 14. 16. 21. 11. 19. 15.  9. 18.  8.  7. 12.  4.
  3.  6.  5.  2.  1.]
ALTERSKATEGORIE_FEIN has value [nan 21. 17. 13. 14. 10. 16. 20. 11. 19. 15. 18.  9. 22. 12.  0.  8.  7.
 23.  4. 24.  6.  3.  2.  5. 25.  1.]
ANZ_HH_TITEL has value [nan  0.  1.  5.  2.  3.  7.  4.  6.  9. 15. 14.  8. 11. 10. 12. 13. 20.
 16. 17. 23. 18.]
ANZ_KINDER has value [nan  0.  1.  2.  3.  4.  5.  6.  9.  7. 11.  8.]
ANZ_PERSONEN has value [nan  2.  1.  0.  4.  3.  5.  7.  6.  8. 12.  9. 21. 10. 13. 11. 14. 45.
 20. 31. 29. 37. 16. 22. 15. 23. 18. 35. 17. 40. 38.]
CAMEO_DEU_2015 has value [nan '8A' '4C' '2A' '6B' '8C' '4A' '2D' '1A' '1E' '9D' '5C' '8B' '7A' '5D'
 '9E' '9B' '1B' '3D' '4E' '4B' '3C' '5A' '7B' '9A' '6D' '6E' '2C' '7C'
 '9C' '7D' '5E' '1D' '8D' '6C' '6A' '5B' '4D' '3A' '2B' '7E' '3B' '6F'
 '5F' '1C']
CAMEO_DEUG_2015 has value [nan '8' '4' '2' '6' '1

In [39]:
azdias.fillna(88, inplace=True)
azdias

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910215,-1,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,3,1,2
910220,-1,9.0,0.0,21.0,0.0,0.0,2.0,0.0,3.0,6.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
910225,-1,9.0,17.0,17.0,0.0,0.0,1.0,0.0,3.0,2.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
910226,2,1.0,13.0,13.0,0.0,0.0,0.0,0.0,2.0,4.0,...,0.0,7.0,10.0,11.0,88.0,9.0,7.0,3,2,4
910241,-1,1.0,20.0,14.0,0.0,0.0,4.0,0.0,4.0,2.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825761,-1,5.0,17.0,17.0,0.0,0.0,1.0,0.0,88.0,6.0,...,2.0,4.0,7.0,8.0,3.0,4.0,3.0,4,2,3
825771,-1,9.0,16.0,16.0,0.0,0.0,1.0,0.0,4.0,7.0,...,4.0,6.0,9.0,6.0,6.0,9.0,5.0,6,1,2
825772,-1,1.0,17.0,17.0,0.0,0.0,0.0,0.0,2.0,5.0,...,2.0,7.0,10.0,8.0,88.0,5.0,7.0,4,2,2


In [40]:
azdias.replace(-1, 88, inplace=True)
azdias

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910215,88,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,3,1,2
910220,88,9.0,0.0,21.0,0.0,0.0,2.0,0.0,3.0,6.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
910225,88,9.0,17.0,17.0,0.0,0.0,1.0,0.0,3.0,2.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
910226,2,1.0,13.0,13.0,0.0,0.0,0.0,0.0,2.0,4.0,...,0.0,7.0,10.0,11.0,88.0,9.0,7.0,3,2,4
910241,88,1.0,20.0,14.0,0.0,0.0,4.0,0.0,4.0,2.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825761,88,5.0,17.0,17.0,0.0,0.0,1.0,0.0,88.0,6.0,...,2.0,4.0,7.0,8.0,3.0,4.0,3.0,4,2,3
825771,88,9.0,16.0,16.0,0.0,0.0,1.0,0.0,4.0,7.0,...,4.0,6.0,9.0,6.0,6.0,9.0,5.0,6,1,2
825772,88,1.0,17.0,17.0,0.0,0.0,0.0,0.0,2.0,5.0,...,2.0,7.0,10.0,8.0,88.0,5.0,7.0,4,2,2


In [41]:
firstPart=azdias[:10]
firstPart

,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTERSKATEGORIE_FEIN,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_TITEL,ARBEIT,BALLRAUM,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
LNR,,,,,,,,,,,,,,,,,,,,,
910215,88,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,3,1,2
910220,88,9.0,0.0,21.0,0.0,0.0,2.0,0.0,3.0,6.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
910225,88,9.0,17.0,17.0,0.0,0.0,1.0,0.0,3.0,2.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
910226,2,1.0,13.0,13.0,0.0,0.0,0.0,0.0,2.0,4.0,...,0.0,7.0,10.0,11.0,88.0,9.0,7.0,3,2,4
910241,88,1.0,20.0,14.0,0.0,0.0,4.0,0.0,4.0,2.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3
910244,3,1.0,10.0,10.0,0.0,0.0,1.0,0.0,2.0,6.0,...,2.0,10.0,7.0,4.0,6.0,9.0,7.0,4,2,1
910248,88,9.0,0.0,88.0,0.0,0.0,1.0,0.0,4.0,6.0,...,2.0,7.0,10.0,10.0,3.0,9.0,5.0,4,2,2
910261,88,1.0,14.0,14.0,0.0,0.0,1.0,0.0,2.0,2.0,...,2.0,10.0,12.0,9.0,5.0,9.0,1.0,1,1,1
645145,88,9.0,16.0,16.0,1.0,0.0,1.0,0.0,2.0,3.0,...,4.0,8.0,11.0,8.0,5.0,8.0,1.0,6,1,3


In [48]:
%%time
firstPart = customers
firstPart = firstPart.astype(str)
firstPart.replace(['-1', '88', '88.0'], np.nan, inplace=True)
firstPart.columns.values

In [43]:
dummies = pd.get_dummies(azdias, prefix_sep='__', sparse=True)


In [44]:
firstPart.shape

(891221, 350)

In [47]:
%%time
dummies.to_pickle("azdias_dummies.pkl")

Wall time: 12.5 s


In [ ]:
for i in dummies.columns:
    print(i)

In [ ]:
azdias.columns.values

In [ ]:
azdias.fillna('88').astype(np.int16).astype(str).applymap(lambda x: np.nan if x == '88' else x)

In [ ]:
azdias.drop(columns=object_dtype_columns, axis=1, inplace=True, errors='ignore')

def wrapValuesToIntAndKeepNaN(dataFrame):
    for col in azdias.columns.values:
        azdias[col].fillna(88).astype(np.int16).astype(str).apply(lambda x: np.nan if x == '88' else x)

In [ ]:
dummies = pd.get_dummies(azdias, prefix_sep='__', sparse=True)
azdias.drop(columns=object_columns, inplace=True)
dummnies.columns.values

## 第1部分：顾客分类报告

项目报告的主体部分应该就是这部分。在这个部分，你应该使用非监督学习技术来刻画公司已有顾客和德国一般人群的人口统计数据的关系。这部分做完后，你应该能够描述一般人群中的哪一类人更可能是邮购公司的主要核心顾客，哪些人则很可能不是。

## 第2部分：监督学习模型

你现在应该已经发现哪部分人更可能成为邮购公司的顾客了，是时候搭建一个预测模型了。"MAILOUT"数据文件的的每一行表示一个邮购活动的潜在顾客。理想情况下我们应该能够使用每个人的人口统计数据来决定是否该把他作为该活动的营销对象。

"MAILOUT" 数据被分成了两个大致相等的部分，每部分大概有 43 000 行数据。在这部分，你可以用"TRAIN"部分来检验你的模型，该数据集包括一列"RESPONSE"，该列表示该对象是否参加了该公司的邮购活动。在下一部分，你需要在"TEST"数据集上做出预测，该数据集中"RESPONSE" 列也被保留了。

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [ ]:
mailout_train = pd.read_csv(drive_path+'Udacity_MAILOUT_052018_TRAIN.csv', sep=';')


In [ ]:
mailout_train.head

## Part 3:Kaggle比赛

你已经搭建了一个用于预测人们有多大程度上会回应邮购活动的模型，是时候到Kaggle上检验一下这个模型了。如果你点击这个 [链接](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140)，你会进入到比赛界面（如果你已经有一个Kaggle账户的话）如果你表现突出的话，你将有机会收到Arvato或Bertelsmann的人力资源管理的经理的面试邀约！

你比赛用提交的文件格式为CSV，该文件含2列。第一列是"LNR"，是"TEST"部分每个顾客的ID。第二列是"RESPONSE"表示此人有多大程度上会参加该活动，可以是某种度量，不一定是概率。你应该在第2部分已经发现了，该数据集存在一个巨大的输出类不平衡的问题，也就是说大部分人都不会参加该邮购活动。因此，预测目标人群的分类并使用准确率来衡量不是一个合适的性能评估方法。相反地，该项竞赛使用AUC衡量模型的性能。"RESPONSE"列的绝对值并不重要：仅仅表示高的取值可能吸引到更多的实际参与者，即ROC曲线的前端曲线比较平缓。

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

```python

```